In [ ]:
import hashlib
from datetime import datetime

genesisMsg = "helloword"
class Block:
    """
    prev_hash:
    data:
    timestamp:    
    hash:
    Nonce: 新增nonce隨機數 以實現pow
    transactions: 交易列表 下方data換成transacation
    """
    def __init__(self, transactions, prev_hash):
        
            self.prev_hash = prev_hash
            self.transactions = transactions
            self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            #設置nonce和hash的初始值為none
            self.nonce = None
            self.hash = None
        
            message = hashlib.sha256()
            message.update(str(self.prev_hash).encode('utf-8'))
            message.update(str(self.transactions).encode('utf-8'))
            message.update(str(self.timestamp).encode('utf-8'))
            self.hash = message.hexdigest()
    def __repr__(self):
        return "區塊內容: %s\n hash值: %s" %(self.transactions, self.hash)
        

In [ ]:
class BlockChain:
    def __init__(self):
        self.blocks = []
        
    def add_block(self,block):
        self.blocks.append(block)

In [ ]:
genesis_block = Block(data="創世區塊",prev_hash ="")

In [ ]:
new_block = Block(data = "shou transfer 1 bit to peggy", prev_hash = genesis_block.hash)
new_block2 = Block(data = "peggy transfer 2 bit to lilian", prev_hash = new_block.hash) 

In [ ]:
blockchain = BlockChain()

blockchain.add_block(genesis_block)
blockchain.add_block(new_block)
blockchain.add_block(new_block2)

In [ ]:
#test
print("區塊數量:%d\n" %len(blockchain.blocks))
      
for block in blockchain.blocks:
    print("父區塊hash: %s" %block.prev_hash)
    print("區塊data: %s" %block.data)
    print("區塊hash: %s" %block.hash)
    print("\n")

In [ ]:
#工作量證明POW
import json
class POW:
    def __init__(self,block,miner,difficult=5):
        self.block = block
        self.difficulty = difficult
        self.reward_amount = 1
        
    #挖礦函數    
    def mine(self):
        i = 0
        prefix = '0' * self.difficulty
        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode('utf-8'))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1
        #獎勵機制
        t = Transaction(sender =  "",
                       recipient = self.miner.address,
                       amount = self.reward_amount,
                       )
        sig = self.miner.sign(json.dumps(t, cls = TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)
    #驗證有效性        
    def validate(self):
        message = hashlib.sha256()
        message.update(str(self.block.prev_hash).encode('utf-8'))
        message.update(str(self.block.transactions).encode('utf-8'))
        message.update(str(self.block.timestamp).encode('utf-8'))
        message.update(str(self.block.nonce).encode('utf-8'))
        digest = message.hexdigest()
        
        prefix = "0" * self.difficulty
        return digest.startswith(prefix)
    
    
                                

In [ ]:
b = Block(data = "test", prev_hash="")
w = POW(b)

In [ ]:
%time valid_block = w.mine()

In [ ]:
%time w.validate()

In [ ]:
blockchain2 = BlockChain()

new_block1 = Block(data = "創世區塊",prev_hash = "")
w1 = POW(new_block1)
genesis_block = w1.mine()
blockchain2.add_block(genesis_block)

new_block2 = Block(data = "shou transfer 2 eth to peggy", prev_hash = genesis_block.hash)
w2 = POW(new_block2)
new_block = w2.mine()
blockchain2.add_block(new_block)

new_block3 = Block(data = "peggy transfer 3 eth to lilian", prev_hash = new_block.hash)
w3 = POW(new_block3)
new_block = w3.mine()
blockchain2.add_block(new_block)

In [ ]:
print("區塊數量:%d\n" %len(blockchain2.blocks))
      
for block in blockchain2.blocks:
    print("父區塊hash: %s" %block.prev_hash)
    print("區塊data: %s" %block.data)
    print("區塊hash: %s" %block.hash)
    print("\n")

In [ ]:
!pip install ecdsa

In [ ]:
#導入橢圓曲線演算法
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import base64
import binascii

class Wallet:
    
    def __init__(self):
        #當錢包初始化時，基於橢圓曲線生成一個唯一的密鑰，且這代表鏈上一個不可重複的帳戶
        self._private_key = SigningKey.generate(curve = SECP256k1) 
        self._public_key = self._private_key.get_verifying_key()
    @property
    def address(self):
    #公鑰生成
        h = hashlib.sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())

    @property
    def pubkey(self):
    #返回公鑰字串
        return self._public_key.to_pem()

    def sign(self, message):
        h = hashlib.sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))

    def verify_sign(pubkey,message,signature):
        verifier = VerifyingKey.from_pem(pubkey)
        h = hashlib.sha256(message.encode('utf8'))
        return verifier.verify(binascii.unhexlify(signature),h.digest())

In [ ]:
w = Wallet()


In [ ]:
w.address

In [ ]:
key = w.pubkey
print(key)

In [ ]:
#test
data = 'helloworld'
sig = w.sign(data)
print(sig)

In [ ]:
w.verify_sign(key,data,sig)

In [ ]:
import json

class Transaction:
    def __init__(self,sender,recipient, amount):
        #初始化交易
        if isinstance(sender,bytes):
            sender = sender.decode('utf-8')
        self.sender = sender  #發送方
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient  #接收方
        self.amount = amount     #數量
        
    def set_sign(self,signature,pubkey):
        self.signature = signature
        self.pubkey=  pubkey
    def __repr__(self):
        if self.sender:
            s = '從%s轉至%s %d個加密貨幣' %(self.sender, self.recipient, self.amount)
        else:
            s = '%s挖礦獲取%d個加密貨幣' %(self.recipient, self.amount)
        return s

In [ ]:
#工作量證明POW
import json
class POW:
    def __init__(self,block,miner,difficult=5):
        self.block = block
        self.difficulty = difficult
        self.reward_amount = 1
        
    #挖礦函數    
    def mine(self):
        i = 0
        prefix = '0' * self.difficulty
        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode('utf-8'))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1
        #獎勵機制
        t = Transaction(sender =  "",
                        ecipient = self.miner.address,
                        amount = self.reward_amount,)
        sig = self.miner.sign(json.dumps(t, cls = TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)
    #驗證有效性        
    def validate(self):
        message = hashlib.sha256()
        message.update(str(self.block.prev_hash).encode('utf-8'))
        message.update(str(self.block.transactions).encode('utf-8'))
        message.update(str(self.block.timestamp).encode('utf-8'))
        message.update(str(self.block.nonce).encode('utf-8'))
        digest = message.hexdigest()
        
        prefix = "0" * self.difficulty
        return digest.startswith(prefix)
    
    
                                

In [ ]:
def get_balance(user):
    balance = 0
    for block in blockchain.blocks:
        for t in block.transactions:
            if t.sender == user.address.decode():
                balance -= t.amount
            elif t.recipient == user.address.decode():
                balance += t.amount
    return balance

In [ ]:
blockchain = BlockChain()

alice = Wallet()
tom = Wallet()
bob = Wallet()

print("alice:%d個加密貨幣" %(get_balance(alice)))
print("tom  :%d個加密貨幣" %(get_balance(tom)))
print("bob  :%d個加密貨幣" %(get_balance(bob)))

In [ ]:
#生成區塊
new_block1 = Block(transactions = [],prev_hash="")
w1 = POW(new_block1, alice)
genesis_block = w1.mine()
blockchain.add_block(genesis_block)

In [ ]:
print("alice:%d個加密貨幣" %(get_balance(alice)))

In [ ]:
transactions = []
new_transaction = Transaction(
    sender = alice.address,
    recipient = tom.address,
    amount = 0.3)
sig = tom.sign(str(new_transaction))
new_transaction.set_sign(sig, tom.pubkey)

In [ ]:
#由bob驗證

if  Wallet.verify_sign(new_transaction.pubkey,str(new_transaction),new_transaction.signature):
    #驗證交易確認，產生新區塊
    print("驗證交易成功")
    new_block2 = Block(transactions = [new_transaction], prev_hash = '')
    print("正在生成新區塊...")
    w2 = POW(new_block2,bob)
    block = w2.mine()
    print("將新區塊加入區塊鏈中")
    blockchain.add_block(block)
else:
    print("交易驗證失敗!")

In [ ]:
print("alice:%d個加密貨幣" %(get_balance(alice)))
print("tom:%d個加密貨幣" %(get_balance(tom)))
print("bob:%d個加密貨幣" %(get_balance(bob)))